In [1]:
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
pd.options.mode.chained_assignment = None  # default='warn'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kalebalemayehu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Synonyms with Thesaurus API

In [2]:
def get_synonyms_api(key_word):
    url = "https://www.dictionaryapi.com/api/v3/references/thesaurus/json/"
    key= "?key=bc125025-b3b8-4b13-b8e8-c1ef37845bad"
    syn_json = requests.get(url+key_word.lower()+key).json()
    syn_list = syn_json[0]['meta']['syns']
    if len(syn_list) > 1:
        master = sum(syn_list, [])
    elif len(syn_list) == 1:
        master = syn_list[0]
    master.append(key_word)
    return master

### Problems with some words

consumer discretionary, consumer staples, communication services (DO NOT WORK)

    -> they do not exist within the Thesaurus, therefore the JSON returns a list with possible words
Definition of Consumer Discretionary:
- goods that are non-essential but desirable if their income is sufficient to purchase them

Definition of Consumer Staples:
- goods that are essential

Definition of Communication Services:
- elecommunications Services, Cable Services, Video Services, or Information Services

-> Solution for now: remove those words from here

In [3]:
df = pd.read_csv("master_taxonomy.csv", index_col=0)#, on_bad_lines='skip')
df = df.reset_index(drop=True)
df.columns

Index(['Authors', 'Title', 'DOI', 'Link', 'Abstract', 'Author Keywords',
       'Index Keywords'],
      dtype='object')

In [4]:
df['Abstract'] = df['Abstract'].str.lower()
df['Title'] = df['Title'].str.lower()
df['Author Keywords'] = df['Author Keywords'].str.lower()
df['Index Keywords'] = df['Index Keywords'].str.lower()

In [5]:
df = df.fillna("blank")

In [6]:
non_alp = [';', ',', '(', ')', '[', ']', '.']
for i in non_alp:
    for col in ['Title','Abstract', 'Author Keywords', 'Index Keywords']:
        df[col] = df[col].map(lambda x: x.replace(i,''))

In [7]:
#df['content_sum'] = df['Abstract'] + df['Title'] + df['Author Keywords'] + df['Index Keywords']
#df["content_sum"] = df["content_sum"].str.lower()
stop = stopwords.words('english')
for i in [df['Abstract'],df['Title'], df['Author Keywords'], df['Index Keywords']]:
    i = i.apply(lambda x: " ".join([word for word in x.split() if word not in (stop)]))

In [8]:
keys = ["energy", "materials", "industrial", "financial",  "utilities"]
no_keys = [ "consumer discretionary", "consumer staples", "information technology", "communication services", "healthcare", "real estate"]

In [9]:
# Saving into a dictionary all of the syns
keys_dict = {}
for i in keys:
    keys_dict[i] = get_synonyms_api(i)
 
#keys_dict

In [10]:

keys_dict["consumer discretionary"] = ["consumer discretionary","non-essential goods", "non essential goods", "non-essential products", "non essential products", "unrestricted goods", "unrestricted products", "nonobligatory goods", "nonobligatory products"]
keys_dict["consumer staples"] = ["essential goods", "essential goods", "essential products", "essential products", "restricted goods", "restricted products", "obligatory goods", "obligatory products"]
keys_dict["information technology"] = ["IT", "information techonology", "i.t", "communication science", "communication sciences", "computer science", "info tech", "cybernetics"]
keys_dict["communication services"] = ["call service", "communication department",  "communication facilities", "communication service","communications department","communications facilities", "communications service","communications services", "facilities-based", "telecommunications service","media service","reporting services", "telecom service", "telecom services", "telecommunication service","telecommunication services"]
keys_dict["healthcare"] = ["medical care","healthcare","medical treatment","medical assistance","medical attention","medical aid","health protection","health maintenance","medical attendance","medical help","medical management","medical service","preventive medicine","wellness program","health management","curative care","public health","maintenance of health","health care service","primary care","medical services","care insurance","health insurance","medical coverage","medical insurance"]
keys_dict["real estate"] = ["building","buildings","housing","immovable","land","motionless","premises","Properties","Property","realtor"]

In [18]:
stage_dict = {}
stage_dict["Ante-hoc"] = ["fuzzy","treebased", "Classification and Regression Tree", "CART Conditional Inference Tree", "CTree", "Decision Tree", "Fast and Frugal Trees", "FFTs", "Fuzzy Hoeffding Decision Tree", "FHDT", "J48", "One-Class Tree", "OCTree", "Multi Operator Temporal Decision Tree", "MTDT", "Recursive Partitioning and Regression Trees (RPART)", "Big Bang–Big Crunch Interval Type-2 Fuzzy Logic System", "BB-BC IT2FLS"]
stage_dict["Post-hoc"] = ["support vector machines", 'neural network','model-agnostic','model-specific','ensemble methods','SVM with Linear and Radial Basis Function (RBF) Kernels','ApparentFlow-net', 'Convolutional Neural Network (CNN)','Adaptive Boosting (AdaBoost)']

In [20]:
for term in stage_dict.keys():
    df[term] = 0
    for wrd in stage_dict[term]:
        for col in [df['Abstract'], df['Title'], df['Author Keywords'], df['Index Keywords']]:
            if col is df['Title']:
                df[term] += col.str.count(wrd) + 0.0004
            elif col is df['Author Keywords']:
                #df[key] += 0.00003
                df[term] += col.str.count(wrd) + 0.0003
            elif col is df['Index Keywords']:
                #df[key] += 0.00002
                df[term] += col.str.count(wrd)  + 0.0002
            else:
                #df[key] += 0.00001
                df[term] += col.str.count(wrd) + 0.0001
df

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,energy,materials,industrial,...,utilities,consumer discretionary,consumer staples,information technology,communication services,healthcare,real estate,Ante-hoc,post-hoc,Post-hoc
0,"Abadía J.J.P., Fritz H., Dadoulis G., Dragos K...",automated decision making in structural health...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the need for processing large amounts of data ...,blank,artificial intelligence damage detection decis...,11.0196,0.0048,5.0056,...,1.0036,0.0036,0.0032,8.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036
1,"Abbass H.A., Hunjet R.A.",smart shepherding: towards transparent artific...,10.1007/978-3-030-60898-9_1,https://www.scopus.com/inward/record.uri?eid=2...,the aim of this chapter is to uncover the beau...,explainable artificial intelligence interpreta...,blank,6.0196,0.0048,5.0056,...,0.0036,0.0036,0.0032,10.0032,0.0064,0.01,1.004,0.0072,0.0036,0.0036
2,"Abdollahi A., Pradhan B.",urban vegetation mapping from aerial imagery u...,10.3390/s21144738,https://www.scopus.com/inward/record.uri?eid=2...,urban vegetation mapping is critical in many a...,deep neural network remote sensing shap vegeta...,aerial photography antennas biodiversity deep ...,2.0196,0.0048,1.0056,...,2.0036,0.0036,0.0032,9.0032,0.0064,0.01,4.004,0.0072,4.0036,4.0036
3,"Abdul A., Von Der Weth C., Kankanhalli M., Lim...",cogam: measuring and moderating cognitive load...,10.1145/3313831.3376615,https://www.scopus.com/inward/record.uri?eid=2...,interpretable machine learning models trade -o...,cognitive load explainable artificial intellig...,computation theory economic and social effects...,5.0196,0.0048,1.0056,...,2.0036,0.0036,0.0032,4.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036
4,"Abe T., Furukawa R., Iwasaki Y., Ikemura T.",time-series trend of pandemic sars-cov-2 varia...,10.5334/dsj-2021-029,https://www.scopus.com/inward/record.uri?eid=2...,to confront the global threat of coronavirus d...,batch-learning self-organizing map blsom covid...,conformal mapping diseases genes machine learn...,15.0196,0.0048,0.0056,...,1.0036,0.0036,0.0032,3.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4586,[No author name available],icmlsc 2021 - proceedings of the 2021 5th inte...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 32 papers the topics d...,blank,blank,7.0196,0.0048,0.0056,...,0.0036,0.0036,0.0032,4.0032,0.0064,0.01,0.004,1.0072,0.0036,0.0036
4587,[No author name available],2021 ieee 29th international conference on net...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 75 papers the topics d...,blank,blank,6.0196,0.0048,0.0056,...,0.0036,0.0036,0.0032,7.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036
4588,[No author name available],10th international conference on computational...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 33 papers the special ...,blank,blank,12.0196,0.0048,0.0056,...,2.0036,0.0036,0.0032,8.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036
4589,[No author name available],proceedings of the 2nd international conferenc...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 21 papers the topics d...,blank,blank,4.0196,0.0048,0.0056,...,0.0036,0.0036,0.0032,3.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036


In [13]:
for key in keys_dict.keys():
    df[key] = 0
    for phr in keys_dict[key]:
        for col in [df['Abstract'], df['Title'], df['Author Keywords'], df['Index Keywords']]:
            if col is df['Title']:
                df[key] += col.str.count(phr) + 0.0004
            elif col is df['Author Keywords']:
                #df[key] += 0.00003
                df[key] += col.str.count(phr) + 0.0003
            elif col is df['Index Keywords']:
                #df[key] += 0.00002
                df[key] += col.str.count(phr)  + 0.0002
            else:
                #df[key] += 0.00001
                df[key] += col.str.count(phr) + 0.0001
df

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,energy,materials,industrial,financial,utilities,consumer discretionary,consumer staples,information technology,communication services,healthcare,real estate
0,"Abadía J.J.P., Fritz H., Dadoulis G., Dragos K...",automated decision making in structural health...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the need for processing large amounts of data ...,blank,artificial intelligence damage detection decis...,11.0196,0.0048,5.0056,0.0024,1.0036,0.0036,0.0032,8.0032,0.0064,0.01,0.004
1,"Abbass H.A., Hunjet R.A.",smart shepherding: towards transparent artific...,10.1007/978-3-030-60898-9_1,https://www.scopus.com/inward/record.uri?eid=2...,the aim of this chapter is to uncover the beau...,explainable artificial intelligence interpreta...,blank,6.0196,0.0048,5.0056,0.0024,0.0036,0.0036,0.0032,10.0032,0.0064,0.01,1.004
2,"Abdollahi A., Pradhan B.",urban vegetation mapping from aerial imagery u...,10.3390/s21144738,https://www.scopus.com/inward/record.uri?eid=2...,urban vegetation mapping is critical in many a...,deep neural network remote sensing shap vegeta...,aerial photography antennas biodiversity deep ...,2.0196,0.0048,1.0056,0.0024,2.0036,0.0036,0.0032,9.0032,0.0064,0.01,4.004
3,"Abdul A., Von Der Weth C., Kankanhalli M., Lim...",cogam: measuring and moderating cognitive load...,10.1145/3313831.3376615,https://www.scopus.com/inward/record.uri?eid=2...,interpretable machine learning models trade -o...,cognitive load explainable artificial intellig...,computation theory economic and social effects...,5.0196,0.0048,1.0056,0.0024,2.0036,0.0036,0.0032,4.0032,0.0064,0.01,0.004
4,"Abe T., Furukawa R., Iwasaki Y., Ikemura T.",time-series trend of pandemic sars-cov-2 varia...,10.5334/dsj-2021-029,https://www.scopus.com/inward/record.uri?eid=2...,to confront the global threat of coronavirus d...,batch-learning self-organizing map blsom covid...,conformal mapping diseases genes machine learn...,15.0196,0.0048,0.0056,0.0024,1.0036,0.0036,0.0032,3.0032,0.0064,0.01,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4586,[No author name available],icmlsc 2021 - proceedings of the 2021 5th inte...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 32 papers the topics d...,blank,blank,7.0196,0.0048,0.0056,0.0024,0.0036,0.0036,0.0032,4.0032,0.0064,0.01,0.004
4587,[No author name available],2021 ieee 29th international conference on net...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 75 papers the topics d...,blank,blank,6.0196,0.0048,0.0056,0.0024,0.0036,0.0036,0.0032,7.0032,0.0064,0.01,0.004
4588,[No author name available],10th international conference on computational...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 33 papers the special ...,blank,blank,12.0196,0.0048,0.0056,0.0024,2.0036,0.0036,0.0032,8.0032,0.0064,0.01,0.004
4589,[No author name available],proceedings of the 2nd international conferenc...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 21 papers the topics d...,blank,blank,4.0196,0.0048,0.0056,0.0024,0.0036,0.0036,0.0032,3.0032,0.0064,0.01,0.004


In [15]:
''''
for key in keys_dict.keys():
    df[key] = 0
    for phr in keys_dict[key]:
        for col in [df['Abstract'], df['Title'], df['Author Keywords'], df['Index Keywords']]:
            if col is df['Title']:
                df[key] += 0.00004
                df[key] += col.str.count(phr)
            elif col is df['Author Keywords']:
                df[key] += 0.00003
                df[key] += col.str.count(phr) 
            elif col is df['Index Keywords']:
                df[key] += 0.00002
                df[key] += col.str.count(phr) 
            else:
                df[key] += 0.00001
                df[key] += col.str.count(phr) 
df
'''

"'\nfor key in keys_dict.keys():\n    df[key] = 0\n    for phr in keys_dict[key]:\n        for col in [df['Abstract'], df['Title'], df['Author Keywords'], df['Index Keywords']]:\n            if col is df['Title']:\n                df[key] += 0.00004\n                df[key] += col.str.count(phr)\n            elif col is df['Author Keywords']:\n                df[key] += 0.00003\n                df[key] += col.str.count(phr) \n            elif col is df['Index Keywords']:\n                df[key] += 0.00002\n                df[key] += col.str.count(phr) \n            else:\n                df[key] += 0.00001\n                df[key] += col.str.count(phr) \ndf\n"

In [92]:
df[df["materials"] > 8]

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,energy,materials,industrial,financial,utilities,consumer discretionary,consumer staples,information technology,communication services,healthcare,real estate
3720,"Hoffmann N., Cerqueira T.F.T., Schmidt J., Mar...",superconductivity in antiperovskites,10.1038/s41524-022-00817-4,https://www.scopus.com/inward/record.uri?eid=2...,we present a comprehensive theoretical study o...,blank,machine learning perturbation techniques super...,12.00196,12.00048,0.00056,0.00024,0.00036,0.00036,0.00032,4.00032,0.00064,0.001,0.0004
3838,"Liu C., Meerten Y., Declercq K., Gryllias K.",vibration-based gear continuous generating gri...,10.1016/j.jmapro.2022.04.068,https://www.scopus.com/inward/record.uri?eid=2...,continuous generating grinding plays an essent...,convolutional neural network deep learning gea...,chemical activation convolutional neural netwo...,8.00196,9.00048,2.00056,0.00024,1.00036,0.00036,0.00032,6.00032,0.00064,0.001,0.0004
4051,"Sun H., Wang C., Cao X.",an adaptive anti-noise gear fault diagnosis me...,10.1016/j.asoc.2022.109120,https://www.scopus.com/inward/record.uri?eid=2...,deep learning networks are widely used to real...,frequency slice wavelet transform gear fault d...,classification of information failure analysis...,1.00196,8.00048,0.00056,0.00024,1.00036,0.00036,0.00032,6.00032,0.00064,0.001,0.0004
4127,"Weng B., Song Z., Zhu R., Yan Q., Sun Q., Gric...",simple descriptor derived from symbolic regres...,10.1038/s41467-020-17263-9,https://www.scopus.com/inward/record.uri?eid=2...,symbolic regression sr is an approach of inter...,blank,perovskite catalyst data set machine learning ...,13.00196,8.00048,0.00056,0.00024,1.00036,0.00036,0.00032,4.00032,0.00064,0.001,1.0004
4192,"Zhang L., Hu W., He M., Xu K., Pan Z.",interpretable machine learning for investigati...,10.1021/acs.jpcc.2c00859,https://www.scopus.com/inward/record.uri?eid=2...,halide perovskite materials exhibit poor optoe...,blank,chemical analysis machine learning perovskite ...,18.00196,9.00048,0.00056,0.00024,0.00036,0.00036,0.00032,10.00032,0.00064,0.001,0.0004
4402,"Liu K., Sadoune N., Rao N., Greitemann J., Pol...",revealing the phase diagram of kitaev material...,10.1103/PhysRevResearch.3.023016,https://www.scopus.com/inward/record.uri?eid=2...,kitaev materials are promising materials for h...,blank,liquids magnets phase diagrams support vector ...,16.00196,10.00048,0.00056,0.00024,1.00036,0.00036,0.00032,5.00032,0.00064,0.001,0.0004
4512,"Velasco L., Castillo J.S., Kante M.V., Olaya J...",phase–property diagrams for multicomponent oxi...,10.1002/adma.202102301,https://www.scopus.com/inward/record.uri?eid=2...,exploring the vast compositional space offered...,high entropy materials high-throughput techniq...,data handling digital libraries energy gap ent...,7.00196,11.00048,1.00056,0.00024,0.00036,0.00036,0.00032,7.00032,0.00064,0.001,0.0004
4566,"Zhang S., Lu T., Xu P., Tao Q., Li M., Lua W.",predicting the formability of hybrid organic-i...,10.1021/acs.jpclett.1c01939,https://www.scopus.com/inward/record.uri?eid=2...,predicting the formability of perovskite struc...,blank,forecasting formability machine learning perov...,11.00196,9.00048,0.00056,0.00024,0.00036,0.00036,0.00032,11.00032,0.00064,0.001,0.0004


In [93]:
sum_dic = {}
for i in keys_dict.keys():
    sum_dic[i] = df[i].sum()
 
sum_dic

{'energy': 28696.99835999983,
 'materials': 581.2036800000001,
 'industrial': 7594.570959999998,
 'financial': 296.1018399999999,
 'utilities': 9684.652759999999,
 'consumer discretionary': 1.6527600000000011,
 'consumer staples': 1.4691200000000006,
 'information technology': 45913.46911999996,
 'communication services': 21.93824000000002,
 'healthcare': 791.5909999999997,
 'real estate': 1802.8364000000008}

In [94]:
df.columns

Index(['Authors', 'Title', 'DOI', 'Link', 'Abstract', 'Author Keywords',
       'Index Keywords', 'energy', 'materials', 'industrial', 'financial',
       'utilities', 'consumer discretionary', 'consumer staples',
       'information technology', 'communication services', 'healthcare',
       'real estate'],
      dtype='object')

In [26]:
df['Taxonomy'] = df[['energy','materials', 'industrial', 'financial', 'utilities',
                    'consumer discretionary', 'consumer staples', 'information technology',
                    'communication services', 'healthcare', 'real estate']].idxmax(axis=1)

In [27]:
df['Stage_Taxonomy'] = df[['Ante-hoc', 'Post-hoc']].idxmax(axis=1)
df

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,energy,materials,industrial,...,consumer staples,information technology,communication services,healthcare,real estate,Ante-hoc,post-hoc,Post-hoc,Stage_Taxonomy,Taxonomy
0,"Abadía J.J.P., Fritz H., Dadoulis G., Dragos K...",automated decision making in structural health...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the need for processing large amounts of data ...,blank,artificial intelligence damage detection decis...,11.0196,0.0048,5.0056,...,0.0032,8.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,energy
1,"Abbass H.A., Hunjet R.A.",smart shepherding: towards transparent artific...,10.1007/978-3-030-60898-9_1,https://www.scopus.com/inward/record.uri?eid=2...,the aim of this chapter is to uncover the beau...,explainable artificial intelligence interpreta...,blank,6.0196,0.0048,5.0056,...,0.0032,10.0032,0.0064,0.01,1.004,0.0072,0.0036,0.0036,Ante-hoc,information technology
2,"Abdollahi A., Pradhan B.",urban vegetation mapping from aerial imagery u...,10.3390/s21144738,https://www.scopus.com/inward/record.uri?eid=2...,urban vegetation mapping is critical in many a...,deep neural network remote sensing shap vegeta...,aerial photography antennas biodiversity deep ...,2.0196,0.0048,1.0056,...,0.0032,9.0032,0.0064,0.01,4.004,0.0072,4.0036,4.0036,Post-hoc,information technology
3,"Abdul A., Von Der Weth C., Kankanhalli M., Lim...",cogam: measuring and moderating cognitive load...,10.1145/3313831.3376615,https://www.scopus.com/inward/record.uri?eid=2...,interpretable machine learning models trade -o...,cognitive load explainable artificial intellig...,computation theory economic and social effects...,5.0196,0.0048,1.0056,...,0.0032,4.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,energy
4,"Abe T., Furukawa R., Iwasaki Y., Ikemura T.",time-series trend of pandemic sars-cov-2 varia...,10.5334/dsj-2021-029,https://www.scopus.com/inward/record.uri?eid=2...,to confront the global threat of coronavirus d...,batch-learning self-organizing map blsom covid...,conformal mapping diseases genes machine learn...,15.0196,0.0048,0.0056,...,0.0032,3.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4586,[No author name available],icmlsc 2021 - proceedings of the 2021 5th inte...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 32 papers the topics d...,blank,blank,7.0196,0.0048,0.0056,...,0.0032,4.0032,0.0064,0.01,0.004,1.0072,0.0036,0.0036,Ante-hoc,energy
4587,[No author name available],2021 ieee 29th international conference on net...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 75 papers the topics d...,blank,blank,6.0196,0.0048,0.0056,...,0.0032,7.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,information technology
4588,[No author name available],10th international conference on computational...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 33 papers the special ...,blank,blank,12.0196,0.0048,0.0056,...,0.0032,8.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,energy
4589,[No author name available],proceedings of the 2nd international conferenc...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 21 papers the topics d...,blank,blank,4.0196,0.0048,0.0056,...,0.0032,3.0032,0.0064,0.01,0.004,0.0072,0.0036,0.0036,Ante-hoc,energy


In [28]:
final_df = df[['Authors', 'Title', 'DOI', 'Link', 'Abstract', 'Author Keywords',
       'Index Keywords', "Taxonomy", "Stage_Taxonomy"]]

In [29]:
final_df

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,Taxonomy,Stage_Taxonomy
0,"Abadía J.J.P., Fritz H., Dadoulis G., Dragos K...",automated decision making in structural health...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the need for processing large amounts of data ...,blank,artificial intelligence damage detection decis...,energy,Ante-hoc
1,"Abbass H.A., Hunjet R.A.",smart shepherding: towards transparent artific...,10.1007/978-3-030-60898-9_1,https://www.scopus.com/inward/record.uri?eid=2...,the aim of this chapter is to uncover the beau...,explainable artificial intelligence interpreta...,blank,information technology,Ante-hoc
2,"Abdollahi A., Pradhan B.",urban vegetation mapping from aerial imagery u...,10.3390/s21144738,https://www.scopus.com/inward/record.uri?eid=2...,urban vegetation mapping is critical in many a...,deep neural network remote sensing shap vegeta...,aerial photography antennas biodiversity deep ...,information technology,Post-hoc
3,"Abdul A., Von Der Weth C., Kankanhalli M., Lim...",cogam: measuring and moderating cognitive load...,10.1145/3313831.3376615,https://www.scopus.com/inward/record.uri?eid=2...,interpretable machine learning models trade -o...,cognitive load explainable artificial intellig...,computation theory economic and social effects...,energy,Ante-hoc
4,"Abe T., Furukawa R., Iwasaki Y., Ikemura T.",time-series trend of pandemic sars-cov-2 varia...,10.5334/dsj-2021-029,https://www.scopus.com/inward/record.uri?eid=2...,to confront the global threat of coronavirus d...,batch-learning self-organizing map blsom covid...,conformal mapping diseases genes machine learn...,energy,Ante-hoc
...,...,...,...,...,...,...,...,...,...
4586,[No author name available],icmlsc 2021 - proceedings of the 2021 5th inte...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 32 papers the topics d...,blank,blank,energy,Ante-hoc
4587,[No author name available],2021 ieee 29th international conference on net...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 75 papers the topics d...,blank,blank,information technology,Ante-hoc
4588,[No author name available],10th international conference on computational...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 33 papers the special ...,blank,blank,energy,Ante-hoc
4589,[No author name available],proceedings of the 2nd international conferenc...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 21 papers the topics d...,blank,blank,energy,Ante-hoc
